In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from utils import *

### Initialize Tagger

In [4]:
from tagger import Tagger
tagger = Tagger()

### Load Model and Data

In [5]:
model_path = '/dfs/scratch0/mccreery/mmtl/logs/best/COLA/logdir/2019_02_25/COLA_20_07_15/'
task_name = 'COLA'
split = 'dev'
filepath = f'{task_name}_{split}_error_analysis.tsv'

In [4]:
# # Load model and data
# model, dl = load_data_and_model(model_path,task_name,split)

# # Create DataFrame of Raw Data, Predictions, and Labels
# print('Creating dataframe')
# df_error = create_dataframe(task_name, model, dl)
# print('Created dataframe')

# # Save (and reload) DataFrame
# csv_path = '/'.join(model_path.split('/')[0:-1])
# save_dataframe(df_error, filepath)

In [6]:
df_error = load_dataframe(filepath)

In [17]:
df_error.loc[df_error['is_wrong']==1][['sentence1', 'label', 'uid']]

sentence1  label  \
4           as you eat the most , you want the least .       0   
5      the more you would want , the less you would ...      0   
13     the more does bill smoke , the more susan hat...      0   
23                               mickey looked up it .       0   
29                       the tube was escaped by gas .       0   
32      what the water did to the bottle was fill it .       0   
33     what the water did to the whole bottle was fi...      0   
34                    the tank leaked the fluid free .       1   
39                 mary beautifully plays the violin .       0   
41                           sue gave to bill a book .       0   
47                   mary intended john to go abroad .       0   
57     i know which book jose didn ' t read for clas...      0   
68     which report that john was inc ##omp ##ete ##...      0   
75     the mayor regarded as being absurd the propos...      0   
76          i want that bill left to remain a secret .       0   
78     drowning cats , which is against the law , ar...      0   
82     the proof this set is rec ##urs ##ive is diff...      0   
83     the mad ##ri ##gal ##s which henry plays the ...      0   
89     i live at the place where route 150 crosses t...      0   
94     which hat did mike qui ##p that she never wor...      0   
97                         i won ' t have some money .       0   
101                i talked to winston about himself .       1   
105    here ' s a knife with which for you to cut up...      0   
108    the younger woman might have been tall and , ...      0   
116    that the cops spoke to the jan ##itor about i...      0   
122    i can ' t remember the name of somebody who h...      0   
123    no writer , and no playwright , meets in vien...      0   
124    no writer , nor any playwright , meets in vie...      0   
132           no one can forgive that comment to you .       0   
160       paperback books lift onto the table easily .       0   
...                                                 ...    ...   
816                   the birds gives the worm a tug .       0   
824          the defendant deny the all ##ega ##tion .       0   
825       the defendants denies the all ##ega ##tion .       0   
844        newsweek appeared on the news ##stand ##s .       1   
845    newsweek about crime appeared on the news ##s...      0   
857                    we preferred them on our team .       1   
864                                     us like them .       0   
870               the sound ##ly and furry cat slept .       0   
877                  nobody told susan about herself .       1   
880                                 i enjoy yourself .       0   
884                                      protect you !       0   
891                      the put ##ter of books left .       0   
897               the horse kicked me black and blue .       1   
899               the horse raced past the barn fell .       1   
913             chris was handed sandy a note by pat .       0   
962             john loaded the wagon with hay green .       0   
967                     how silly is bill considered ?       1   
968                            how mad was bill made ?       1   
972                      john gave bill the dog dead .       1   
980               john was promised by bill to leave .       0   
985    on whom are you counting to inc ##rim ##inate...      1   
990                   it was hated for john to leave .       0   
1001             john ' s arriving dead surprised me .       1   
1004              john left us orders to follow pete .       1   
1009     john regretted it that bill had a good time .       1   
1011    john believes it sincerely that bill is here .       0   
1022   it was believed to be illegal by them to do t...      0   
1025                             i bought it to read .       1   
1037   the correspondence school sent bill a good ty...      1   
1040      john bough

In [19]:
df_error.loc[df_error['uid']=='CoLA/dev.tsv:6']

Unnamed: 0                                          sentence1  sentence2  \
5           5   the more you would want , the less you would ...        NaN   

      score  label             uid  pred  is_wrong  
5  0.914421      0  CoLA/dev.tsv:6     1      True

## Sandbox for Error Analysis

**0. Some basic statistics.**

Confusion Matrix and Performance Metrics

In [8]:
from metal.analysis import confusion_matrix
from metal.utils import convert_labels

Y_gold = convert_labels(df_error['label'].values, "onezero", "categorical")
Y_preds = convert_labels(df_error['pred'].values, "onezero", "categorical")
Y_probs = np.vstack([df_error['score'].values, 1 - df_error['score'].values]).transpose()
confusion_matrix(Y_gold, Y_preds, pretty_print=True, normalize=True)
print()

from metal.metrics import metric_score
metric_list = ['accuracy','precision', 'recall', 'f1']

for metric in metric_list:
    score = metric_score(Y_gold, Y_preds, metric, probs=Y_probs)
    print(f"{metric.capitalize()}: {score:.3f}")

        y=1    y=2   
 l=1  *0.839  0.161  
 l=2   0.190 *0.810  

Accuracy: 0.832
Precision: 0.839
Recall: 0.938
F1: 0.885


Plotting Predictions and Predicted Probabilistic Label Distribution

In [8]:
from metal.contrib.visualization.analysis import (
        plot_predictions_histogram, 
        plot_probabilities_histogram,
        plot_calibration_histogram
    )
plot_predictions_histogram(Y_preds, Y_gold, title="Label Distribution")
plot_calibration_histogram(Y_probs, Y_gold, title="Probablistic Label Distribution", legend=[])

Accuracy:  0.7364620938628159


**1. We want to look at examples that are "barely" wrong and "barely" right since we have hope for boosts here.**

In [15]:
print("\033[1mBARELY WRONG\033[0;0m")
print_barely_wrong(df_error, thresh=0.2, n=3)

BARELY WRONG
30 matches were found with the given criteria.

UID:  CoLA/dev.tsv:726
sentence1: 	 the only person whose kids dana is willing to put up with was written by john . 
sentence2: 	nan
score: 	0.6524
label: 	0

UID:  CoLA/dev.tsv:858
sentence1: 	 we preferred them on our team . 
sentence2: 	nan
score: 	0.4924
label: 	1

UID:  CoLA/dev.tsv:631
sentence1: 	 the magazines were sent to herself by mary . 
sentence2: 	nan
score: 	0.5206
label: 	0



In [10]:
print("\033[1mBARELY RIGHT\033[0;0m")
print_barely_right(df_error, thresh=0.2, n=3)

BARELY RIGHT
7 matches were found with the given criteria.

UID:  RTE/dev.tsv:141
sentence1: 	 los angeles county probation officials say they are now studying how other counties recover juvenile detention costs , after admitting they mistakenly billed parents for days when youths were held in probation camps and halls . by law , california counties can bill parents and legal guardians for some daily costs of det ##ain ##ing youths , but only those whose parents can afford to pay . last year , more than 20 , 000 youths were admitted to probation camps and halls , and l . a . county billed parents a daily charge of $ 11 . 94 for camps , $ 23 . 63 for halls . 
sentence2: 	 in los angeles county all parents have to pay the detention costs of their children . 
score: 	0.4265
label: 	0

UID:  RTE/dev.tsv:16
sentence1: 	 take consumer products giant pro ##cter and gamble . even with a $ 1 . 8 billion research and development budget , it still manages 500 active partnerships each year , many 

**2. We also want to look at examples we got completely wrong since that could point to a systematic bias in the data/model. It could also help us find examples in the dataset that are mislabeled by human annotators**

In [11]:
tagger.add_tag("RTE/dev.tsv:120", 'entity_confusion')

Added 1 tag. Tag set 'entity_confusion' contains 2 tags.


In [12]:
print("\033[1mVERY WRONG\033[0;0m")
print_very_wrong(df_error, thresh=0.9, n=30)

VERY WRONG
69 matches were found with the given criteria.

UID:  RTE/dev.tsv:125
sentence1: 	 a plane crashed in north carolina last year and most of the blink - 182 group was in there . the only survivors of the crash were dj am and musical partner travis barker . less lucky , " lil " chris baker , barker ' s close friend and assistant , the blink - 182 drummer ' s bodyguard , charles " che " still , the pilot and co - pilot died . both am and barker suffered many in ##jure ##s , bad burns and they needed to stay in hospital for several weeks . now am asks $ 20 million in compensation for injuries he suffered . he lawsuits against plane maker lear ##jet and wants to gain $ 10 million for medical damages , lost earnings and profit and the same amount for mental and physical suffering . 
sentence2: 	 travis barker belongs to a band . 
score: 	0.0288
label: 	1

UID:  RTE/dev.tsv:186
sentence1: 	 he said that " there is evidence that cr ##ist ##iani was involved in the murder of the six j

In [13]:
print("\033[1mVERY RIGHT\033[0;0m")
print_very_right(df_error, thresh=0.9, n=3)

VERY RIGHT
188 matches were found with the given criteria.

UID:  RTE/dev.tsv:143
sentence1: 	 a cat ##ac ##ly ##sm ##ic star ##qua ##ke is thought to have caused a flare of radiation that ripped past the earth on december 27 , batter ##ing instruments on satellites and lighting up our atmosphere . 
sentence2: 	 the flash of radiation on december 27 , lit up the earth ' s atmosphere . 
score: 	0.9977
label: 	1

UID:  RTE/dev.tsv:128
sentence1: 	 confident that those grading papers would understand answers written in text - speak , ha ##que stressed that in some exams , including english , text abbreviation ##s would be penal ##ized . 
sentence2: 	 ha ##que wants to include english in some exams . 
score: 	0.0115
label: 	0

UID:  RTE/dev.tsv:71
sentence1: 	 gi ##ulia ##na sg ##ren ##a was freed from captivity in iraq today . the italian journalist was abducted in baghdad exactly a month ago while she had been reporting for il manifesto . the islamic jihad organisation had taken her host

**3. To find systematic errors, we can also look for correlations between certain features and the incorrectness a la Socratic**


We can make this way more sophisticated by perhaps using embeddings instead of this simple [BoW featurization](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer).

In [14]:
print("\033[1mSYSTEMATIC EXAMPLES\033[0;0m")
for i in range(3):
    print_systematic_wrong(df_error)

SYSTEMATIC EXAMPLES
award winning
supreme court said
foreign minister


/dfs/scratch0/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


22 year old
mark antony

8 matches were found with the given criteria.

UID:  RTE/dev.tsv:77
sentence1: 	 a closely divided u . s . supreme court said on thursday its 2002 ruling that ju ##ries and not judges must impose a death sentence applies only to future cases , a decision that may affect more than 100 death row inmates . 
sentence2: 	 the supreme court decided that only judges can impose the death sentence . 
score: 	0.9890
label: 	0

award winning
court said
new government
22 year old
patti nson

8 matches were found with the given criteria.

UID:  RTE/dev.tsv:220
sentence1: 	 egypt on thursday strongly criticized israeli new foreign minister av ##ig ##dor lie ##berman for his remarks that he refused to recognize the peace efforts initiated in 2007 in the u . s . city of annapolis to restore the peace talks with the palestinians , reported the state men ##a news agency . lie ##berman ' s remarks is " regret ##table , " egyptian foreign ministry spokesman ho ##ssa ##m za ##ki wa

# BH LFs

In [30]:
from metal.mmtl.debugging.utils import print_row, apply_lfs_to_df, view_matches
from metal.mmtl.debugging.lf_helpers import regex_present

In [42]:
def LF_accuse(row):
    return 2 if regex_present(row, "accuse") else 0

lfs = [
    LF_accuse,
]

In [45]:
L, Y = apply_lfs_to_df(df_error, lfs)
view_matches(df_error, lfs[0], n=5)

Displaying 5/9 matches

sentence1: 	 he said that " there is evidence that cr ##ist ##iani was involved in the murder of the six jesuit priests " which occurred on 16 november in san salvador . 
sentence2: 	 cr ##ist ##iani is accused of the assassination of six jesuits . 
score: 	0.0042
label: 	1

sentence1: 	 the court in anger ##s handed down sentences ranging from four months suspended to 28 years for , among others , phil ##ppe v . , the key accused . the court found that he , along with his son fran ##ck v . and fran ##ck ' s former spouse , patricia m . , was one the ins ##ti ##gat ##ors of a sex ring that abused 45 children , mostly in the couple ' s flat . the abuses of children aged between six months and 12 years took place in a poor and deprived area of the western french town of anger ##s . many of the defendants were poor and lived on benefits and some were mentally impaired . about 20 of them admitted to the charges , while others claimed to have never heard of a sex rin

In [44]:
from scipy.sparse import csr_matrix
from metal.analysis import lf_summary

Y_cat = convert_labels(Y, "onezero", "categorical")
lf_summary(csr_matrix(L), Y_cat)

Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0       2.0  0.032491       0.0        0.0        5          4   0.555556

## Designing Labeling Functions

**1. Number Based LF**

Our model tends to fail when there are numbers involved in the two sentences. We can look for the same number being repeated in both sentences as an LF

In [ ]:
def LF_number(idx):
    sentence1_nums = [int(s) for s in df_error['sentence1'][idx].split() if s.isdigit()]
    sentence2_nums = [int(s) for s in df_error['sentence2'][idx].split() if s.isdigit()]
    common_nums = len(set(sentence1_nums).intersection(set(sentence2_nums)))
    
    if (sentence1_nums == []) or (sentence2_nums == []):
        return 0
    
    if common_nums > 0:
        return 2
    else:
        return 1

In [ ]:
print_row(df_error.iloc[70])
print("LF_label: ", LF_number(70)-1)

print()
print()
print_row(df_error.iloc[254])
print("LF_label: ", LF_number(254)-1)

**2. Edit Distance Based LF**

Our model tends to vote entailment when one sentence is long and the other is short. We can focus on this slice and flip the labelt o vote no entailment even when the number of words is the same.

In [ ]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

from collections import Counter
def common_words(s1,s2):
    s1_set = set(Counter(s1.split()))
    s2_set = set(Counter(s2.split()))
    return len(s1_set.intersection(s2_set))/float(min(len(s1_set),len(s2_set)))

In [ ]:
def LF_words(idx):
    ratio = common_words(df_error['sentence1'][idx], df_error['sentence2'][idx])
    if ratio < 0.3:
        return 1
    if (ratio <= 1.0) and (ratio > 0.4):
        return 2
    else:
        return 0

In [ ]:
print_row(df_error.iloc[150])
print("LF_label: ", LF_words(150)-1)

print()
print()
print_row(df_error.iloc[95])
print("LF_label: ", LF_words(95)-1)

### Analyze Labeling Functions

In [ ]:
L = np.zeros((np.shape(df_error)[0],2))
for i in range(df_error.shape[0]):
    L[i,0] = LF_number(i)
    L[i,1] = LF_words(i)

Labeling Function Summary

In [ ]:
from metal.analysis import lf_summary
from scipy.sparse import csr_matrix    

L_sparse = csr_matrix(L)
lf_summary(L_sparse,Y=df_error.label+1)

In [ ]:
incorrect = set(np.where(df_error.is_wrong == True)[0])
LF1_set = set(np.where(L[:,0]-1. == df_error.label)[0])
LF2_set = set(np.where(L[:,1]-1. == df_error.label)[0])

print("Percentage Corrected by LF_num: ", 100.*len(LF1_set.intersection(incorrect))/float(len(incorrect)))
print("Percentage Corrected by LF_words: ", 100.*len(LF2_set.intersection(incorrect))/float(len(incorrect)))